IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\FAULT_M6_10


In [3]:
guasto=2

VOLO m6 FAULT 10%

In [4]:
rcou_m6_fault10 = pd.read_csv("RCOU.csv")
rcou_m6_fault10 = rcou_m6_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_fault10 = rcou_m6_fault10[((rcou_m6_fault10['C9'] >= potenza) & (rcou_m6_fault10['C10'] >= potenza) & (rcou_m6_fault10['C11'] >= potenza) & (rcou_m6_fault10['C12'] >= potenza) & (rcou_m6_fault10['C13'] >= potenza) & (rcou_m6_fault10['C14']>= potenza))]
rcou_m6_fault10=rcou_m6_fault10.reset_index(drop=True)
display(rcou_m6_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,95775265,1389,1443,1402,1431,1367,1463
1,95796452,1411,1447,1454,1403,1431,1428
2,95827847,1416,1477,1448,1447,1411,1481
3,95848531,1366,1540,1496,1421,1404,1510
4,95859847,1543,1376,1434,1496,1511,1416
...,...,...,...,...,...,...,...
7526,189014770,1661,1360,1649,1379,1470,1581
7527,189045755,1663,1365,1642,1397,1467,1590
7528,189056149,1594,1461,1661,1367,1402,1638
7529,189090843,1575,1480,1647,1385,1361,1663


In [5]:
min=rcou_m6_fault10['TimeUS'][0]
max=rcou_m6_fault10['TimeUS'][len(rcou_m6_fault10)-1]
print(max)
print(min)

189119715
95775265


In [6]:
xkf1_m6_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m6_fault10 = xkf1_m6_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_fault10 = xkf1_m6_fault10[((xkf1_m6_fault10['TimeUS'] >= min) & (xkf1_m6_fault10['TimeUS'] <= max	))]
xkf1_m6_fault10 = xkf1_m6_fault10.reset_index(drop=True)
print(xkf1_m6_fault10)

         TimeUS  Roll  Pitch    Yaw
0      95783820  0.22   0.84  22.93
1      95793929  0.31   0.82  22.94
2      95804499  0.50   0.78  22.93
3      95815177  0.60   0.83  22.94
4      95825583  0.47   0.89  22.92
...         ...   ...    ...    ...
8479  189064391 -0.35  -1.60   8.61
8480  189078234 -0.40  -1.70   8.56
8481  189088483 -0.54  -1.71   8.55
8482  189098837 -0.47  -1.83   8.53
8483  189117546 -0.47  -1.76   8.44

[8484 rows x 4 columns]


In [7]:
att_m6_fault10 = pd.read_csv("ATT.csv")
att_m6_fault10 = att_m6_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_fault10 = att_m6_fault10[((att_m6_fault10['TimeUS'] >= min) & (att_m6_fault10['TimeUS'] <= max	))]
att_m6_fault10=att_m6_fault10.reset_index(drop=True)
print(att_m6_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      95785119    -0.07  0.22     -0.24   0.84   22.79  22.93
1      95795803    -0.10  0.31     -0.25   0.82   22.79  22.94
2      95806244    -0.12  0.50     -0.27   0.78   22.79  22.93
3      95816755    -0.13  0.60     -0.29   0.83   22.79  22.94
4      95826942    -0.14  0.47     -0.32   0.89   22.79  22.92
...         ...      ...   ...       ...    ...     ...    ...
8480  189065714     1.84 -0.35     -1.66  -1.60    3.38   8.61
8481  189079347     1.81 -0.40     -1.63  -1.70    3.38   8.56
8482  189089980     1.80 -0.54     -1.61  -1.71    3.38   8.55
8483  189100791     1.78 -0.47     -1.59  -1.83    3.38   8.53
8484  189118903     1.75 -0.47     -1.57  -1.76    3.39   8.44

[8485 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m6_fault10)>len(xkf1_m6_fault10):
    to_add=att_m6_fault10[len(xkf1_m6_fault10):]
    att_m6_fault10=att_m6_fault10[:len(xkf1_m6_fault10)]

if len(xkf1_m6_fault10)>len(att_m6_fault10):
    to_add=xkf1_m6_fault10[len(att_m6_fault10):]

for idx,i in enumerate(att_m6_fault10['Roll']):
    if(xkf1_m6_fault10['Roll'][idx] != i):
        att_m6_fault10['Roll'][idx] = ((att_m6_fault10['Roll'][idx] + xkf1_m6_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m6_fault10['Pitch']):
    if(xkf1_m6_fault10['Pitch'][idx] != i):
        att_m6_fault10['Pitch'][idx] = ((att_m6_fault10['Pitch'][idx] + xkf1_m6_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_fault10['Yaw']):
    if(xkf1_m6_fault10['Yaw'][idx] != i):
        att_m6_fault10['Yaw'][idx] = ((att_m6_fault10['Yaw'][idx] + xkf1_m6_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_fault10=pd.concat([att_m6_fault10,to_add])

print(att_m6_fault10)

C:\Users\chiar\AppData\Local\Temp\ipykernel_25784\755583426.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m6_fault10['Roll'][idx] = ((att_m6_fault10['Roll'][idx] + xkf1_m6_fault10['Roll'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25784\755583426.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m6_fault10['Pitch'][idx] = ((att_m6_fault10['Pitch'][idx] + xkf1_m6_fault10['Pitch'][idx])/2)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25784\755583426.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0      95785119    -0.07  0.220     -0.24  0.840   22.79  22.930
1      95795803    -0.10  0.310     -0.25  0.820   22.79  22.940
2      95806244    -0.12  0.500     -0.27  0.780   22.79  22.930
3      95816755    -0.13  0.600     -0.29  0.830   22.79  22.940
4      95826942    -0.14  0.470     -0.32  0.890   22.79  22.920
...         ...      ...    ...       ...    ...     ...     ...
8480  189065714     1.84 -0.375     -1.66 -1.650    3.38   8.585
8481  189079347     1.81 -0.470     -1.63 -1.705    3.38   8.555
8482  189089980     1.80 -0.505     -1.61 -1.770    3.38   8.540
8483  189100791     1.78 -0.470     -1.59 -1.795    3.38   8.485
8484  189118903     1.75 -0.470     -1.57 -1.760    3.39   8.440

[8485 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m6_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m6_fault10 = esc_0_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m6_fault10 = esc_0_m6_fault10[((esc_0_m6_fault10['TimeUS'] >= min) & (esc_0_m6_fault10['TimeUS'] <= max))]
esc_0_m6_fault10=esc_0_m6_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m6_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m6_fault10 = esc_1_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m6_fault10 = esc_1_m6_fault10[((esc_1_m6_fault10['TimeUS'] >= min) & (esc_1_m6_fault10['TimeUS'] <= max))]
esc_1_m6_fault10=esc_1_m6_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m6_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m6_fault10 = esc_2_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m6_fault10 = esc_2_m6_fault10[((esc_2_m6_fault10['TimeUS'] >= min) & (esc_2_m6_fault10['TimeUS'] <= max))]
esc_2_m6_fault10=esc_2_m6_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m6_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m6_fault10 = esc_3_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m6_fault10 = esc_3_m6_fault10[((esc_3_m6_fault10['TimeUS'] >= min) & (esc_3_m6_fault10['TimeUS'] <= max))]
esc_3_m6_fault10=esc_3_m6_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m6_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m6_fault10 = esc_4_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m6_fault10 = esc_4_m6_fault10[((esc_4_m6_fault10['TimeUS'] >= min) & (esc_4_m6_fault10['TimeUS'] <= max))]
esc_4_m6_fault10=esc_4_m6_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m6_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m6_fault10 = esc_5_m6_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m6_fault10 = esc_5_m6_fault10[((esc_5_m6_fault10['TimeUS'] >= min) & (esc_5_m6_fault10['TimeUS'] <= max))]
esc_5_m6_fault10=esc_5_m6_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m6_fault10 = imu_0_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_fault10 = imu_0_m6_fault10[((imu_0_m6_fault10['TimeUS'] >= min) & (imu_0_m6_fault10['TimeUS'] <= max))]

imu_1_m6_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m6_fault10 = imu_1_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_fault10 = imu_1_m6_fault10[((imu_1_m6_fault10['TimeUS'] >= min) & (imu_1_m6_fault10['TimeUS'] <= max))]

imu_2_m6_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m6_fault10 = imu_2_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_fault10 = imu_2_m6_fault10[((imu_2_m6_fault10['TimeUS'] >= min) & (imu_2_m6_fault10['TimeUS'] <= max))]

imu_m6_fault10 = pd.concat((imu_0_m6_fault10, imu_1_m6_fault10, imu_2_m6_fault10))
imu_m6_fault10=imu_m6_fault10.groupby(imu_m6_fault10.TimeUS, as_index=False).mean()

display(imu_m6_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,95777043,0.602359,-0.056894,-0.000354,-0.039858,1.590671,-9.782290
1,95780532,0.620342,-0.147095,0.034039,0.125093,0.755188,-9.501524
2,95782925,-0.085625,-0.118474,0.042167,0.185636,-0.309531,-9.536599
3,95784938,-0.370824,0.018751,0.013881,0.083057,-0.263599,-9.828410
4,95787735,-0.001924,0.109419,-0.019396,-0.036998,0.582240,-10.132694
...,...,...,...,...,...,...,...
32248,189100550,0.071910,-0.026657,-0.051577,-0.556881,0.403775,-10.109230
32249,189107000,-0.230953,0.107526,-0.084279,-0.836197,0.923446,-10.389923
32250,189109336,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483
32251,189116807,-0.164212,0.141485,-0.067597,-0.956638,0.871506,-10.755723


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m6_fault10 = pd.merge_ordered(imu_m6_fault10,att_m6_fault10)
m6_fault10=m6_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_0_m6_fault10)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_1_m6_fault10)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_2_m6_fault10)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_3_m6_fault10)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_4_m6_fault10)
m6_fault10 = pd.merge_ordered(m6_fault10, esc_5_m6_fault10)
m6_fault10=m6_fault10.fillna(method="ffill").fillna(method="bfill")
m6_fault10 = pd.merge_ordered(m6_fault10, rcou_m6_fault10)
m6_fault10=m6_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
index_to_remove=[]
for ind in att_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_0_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_1_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_2_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_3_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_4_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in esc_5_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))
print(len(index_to_remove))
for ind in rcou_m6_fault10["TimeUS"]:
    if ind not in index_to_remove:
        index_to_remove.append(int(ind))

8485
9280
10054
10850
11651
12432
13204


In [13]:
m6_fault10 = m6_fault10[~m6_fault10.TimeUS.isin(index_to_remove)]
m6_fault10["TimeUS"] = m6_fault10["TimeUS"] - m6_fault10.iloc[0]["TimeUS"]
m6_fault10["TimeUS"] = m6_fault10["TimeUS"].astype(int)
m6_fault10["Guasto"] = guasto

In [14]:
from datetime import timedelta
m6_fault10=m6_fault10.reset_index(drop=True)
m6_fault10['TimeUS'] = pd.to_datetime(m6_fault10['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.602359,-0.056894,-0.000354,-0.039858,1.590671,-9.782290,-0.07,0.220,-0.24,...,2.95,4112.0,3.05,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2
1,00:00:00.003489,0.620342,-0.147095,0.034039,0.125093,0.755188,-9.501524,-0.07,0.220,-0.24,...,2.95,4112.0,3.05,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2
2,00:00:00.005882,-0.085625,-0.118474,0.042167,0.185636,-0.309531,-9.536599,-0.07,0.220,-0.24,...,2.95,4112.0,3.05,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2
3,00:00:00.007895,-0.370824,0.018751,0.013881,0.083057,-0.263599,-9.828410,-0.07,0.220,-0.24,...,2.95,4112.0,3.05,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2
4,00:00:00.010692,-0.001924,0.109419,-0.019396,-0.036998,0.582240,-10.132694,-0.07,0.220,-0.24,...,2.95,4112.0,3.05,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32248,00:01:33.323507,0.071910,-0.026657,-0.051577,-0.556881,0.403775,-10.109230,1.80,-0.505,-1.61,...,1.78,6307.0,3.99,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2
32249,00:01:33.329957,-0.230953,0.107526,-0.084279,-0.836197,0.923446,-10.389923,1.78,-0.470,-1.59,...,1.78,6307.0,3.99,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2
32250,00:01:33.332293,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483,1.78,-0.470,-1.59,...,1.78,6307.0,3.99,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2
32251,00:01:33.339764,-0.164212,0.141485,-0.067597,-0.956638,0.871506,-10.755723,1.78,-0.470,-1.59,...,1.78,6307.0,3.99,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2


## Analisi nel tempo e in frequenza

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_fault10)/350)):
        V1=m6_fault10[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25784\4199471638.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_25784\4199471638.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.029732,0.018382,3.507550,0.138613,72.0,71.0,7.417970,5.023942,-0.017565,...,9410.462595,3.200513,0.091627,-1.179562,3.214755,109.0,106.0,10.282299,5.960102,2
1,1.0,-0.029561,0.018340,3.513535,0.138425,70.0,0.0,9.055479,4.857794,-0.017351,...,9286.757620,3.200511,0.091366,-1.174296,3.214713,113.0,110.0,9.858950,5.457796,2
2,2.0,-0.029402,0.018297,3.520755,0.138236,65.0,0.0,9.882301,5.015794,-0.017399,...,9109.768948,3.200510,0.091107,-1.169031,3.214671,111.0,109.0,9.431847,5.213848,2
3,3.0,-0.029831,0.018310,3.509927,0.138378,75.0,0.0,9.092978,4.965207,-0.016813,...,8879.735269,3.200508,0.090848,-1.163766,3.214630,103.0,101.0,9.002360,5.028435,2
4,4.0,-0.030105,0.018285,3.518237,0.138347,71.0,70.0,7.552173,5.801660,-0.016249,...,8597.815322,3.200507,0.090592,-1.158502,3.214589,113.0,111.0,8.571823,4.796136,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,87.0,-0.017037,0.017046,2.962112,0.131521,0.0,102.0,16.436231,8.010068,-0.014022,...,7290.223435,3.222534,0.078941,-1.060653,3.234731,137.0,135.0,15.071252,5.697451,2
88,88.0,-0.016900,0.017016,2.966673,0.131387,0.0,94.0,16.527974,8.419833,-0.014208,...,6910.033443,3.222415,0.078767,-1.056199,3.234585,137.0,135.0,15.069996,5.829071,2
89,89.0,-0.016903,0.016977,2.980274,0.131241,0.0,105.0,16.557055,9.780802,-0.014487,...,6507.716422,3.222295,0.078594,-1.051742,3.234440,133.0,131.0,15.065044,5.929882,2
90,90.0,-0.017158,0.016967,2.980870,0.131237,0.0,96.0,16.473537,10.271622,-0.014600,...,6393.617020,3.222177,0.078421,-1.047284,3.234295,141.0,139.0,15.056581,5.996140,2


In [18]:
path_file = path_file.replace(r"M6\FAULT_M6_10", "")
os.chdir(path_file)
df_merged.to_csv('m6_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
